<a href="https://colab.research.google.com/github/G-Gaddu/Quant-Material/blob/main/Warren_Buffett_Stock_Selection_S%26P500.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Warren Buffet's stock selection method used on the S&P500 constituents

In [49]:
# First import the necessary packages
import yfinance as yf
import pandas as pd
import numpy as np
import warnings

In [50]:
# Suppress warnings
warnings.filterwarnings('ignore')


In [51]:
# Create a function to pull the financial data for each stock from yahoofinance
def get_financial_data(ticker_symbol):
    try:
        equity = yf.Ticker(ticker_symbol)
        data = equity.info
        earngs = equity.earnings
        cflow = equity.cashflow

        return {
            'Ticker': ticker_symbol,
            'ROIC': data.get('returnOnInvestment', np.nan),
            'ROE': data.get('returnOnEquity', np.nan),
            'Gross Margin': data.get('grossMargins', np.nan),
            'Revenue Growth': calculate_revenue_growth(earngs),
            'Free Cash Flow': calculate_free_cash_flow(cflow),
            'Debt to Equity': data.get('debtToEquity', np.nan),
            'Earnings Stability': calculate_earnings_stability(earngs)
        }
    except Exception as e:
        print(f"Error fetching data for {ticker_symbol}: {e}")
        return None


In [52]:
# From the earnings determine the revenue growth over the last 5 years
def calculate_revenue_growth(earnings):
    try:
        if len(earnings) >= 5:
            return (earnings['Revenue'].iloc[4] - earnings['Revenue'].iloc[0]) / earnings['Revenue'].iloc[0]
        return np.nan
    except:
        return np.nan

In [53]:
# Calculate Free Cash Flow
def calculate_free_cash_flow(cashflow):
    try:
        return cashflow.loc['Total Cash From Operating Activities'][0] + \
               (cashflow.loc['Capital Expenditures'][0] if 'Capital Expenditures' in cashflow.index else 0)
    except:
        return np.nan

In [55]:
# Calculate Earnings Stability (Standard Deviation of Earnings Growth)
def calculate_earnings_stability(earnings):
    try:
      if len(earnings) >= 5:
          earngs_gwh = earnings['Earnings'].pct_change().dropna()
          if len(earngs_gwh) > 1:
              return earngs_gwh.std()
          return np.nan
      return np.nan
    except:
        return np.nan

In [56]:
# Define scoring functions for each metric
def score_roic(roic):
    return score_metric(roic, [15, 10, 5], [1, 0.7, 0.4])

def score_roe(roe):
    return score_metric(roe, [20, 15, 10], [1, 0.7, 0.4])

def score_gross_margin(gm):
    return score_metric(gm, [50, 40, 30], [1, 0.7, 0.4])

def score_revenue_growth(rg):
    return score_metric(rg, [0.15, 0.10, 0.05], [1, 0.7, 0.4])

def score_free_cash_flow(fcf):
    return 1 if (not pd.isna(fcf)) and fcf > 0 else 0

def score_debt_to_equity(de):
    return score_metric(de, [0.5, 1, 2], [1, 0.7, 0.4])

def score_earnings_stability(es):
    return score_metric(es, [0.05, 0.10, 0.15], [1, 0.7, 0.4])


In [57]:
# Create a function to determine the score by taking an equal weighted average of each score
def compute_moat_score(row):
    scores = [
        score_roic(row['ROIC']),
        score_roe(row['ROE']),
        score_gross_margin(row['Gross Margin']),
        score_revenue_growth(row['Revenue Growth']),
        score_free_cash_flow(row['Free Cash Flow']),
        score_debt_to_equity(row['Debt to Equity']),
        score_earnings_stability(row['Earnings Stability'])
    ]
    # Assign equal weights
    moat_score = np.sum(scores) / len(scores)  # Average score
    return moat_score

# Pull the data for each stock and get the score for them, return the scores

def get_moat_stocks(ticker):
    financial_data = []
    data = get_financial_data(ticker)
    if data:
        financial_data.append(data)

    df = pd.DataFrame(financial_data)
    moat_stock = df.apply(compute_moat_score, axis=1)
    return moat_stock.iloc[0]


In [58]:
# Get the tickers for the S&P 500 constituents
sp500_tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]['Symbol'].tolist()

In [59]:
# From yahoofinance pull the interest coverage ratio for each stock
def get_interest_coverage(ticker):
    try:
        stock = yf.Ticker(ticker)
        income_statement = stock.financials

        # Ensure the necessary data is present
        if 'Operating Income' in income_statement.index and 'Interest Expense' in income_statement.index:
            operating_income = income_statement.loc['Operating Income'][0]
            interest_expense = income_statement.loc['Interest Expense'][0]

            # Handle cases where Interest Expense is zero to avoid division by zero
            if interest_expense != 0:
                interest_coverage = operating_income / abs(interest_expense)
                return round(interest_coverage, 2)
            else:
                print(f"Interest Expense is zero for {ticker}. Cannot compute Interest Coverage Ratio.")
                return None
        else:
            print(f"Necessary financial data not available for {ticker}.")
            return None

    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return None

In [40]:
def get_financials(ticker):
    try:
        stock = yf.Ticker(ticker)
        balance = stock.balance_sheet
        cashflow = stock.cashflow
        income = stock.financials
        info = stock.info

        # Extract necessary metrics
        debt_equity = info.get('debtToEquity', np.nan)
        current_ratio = info.get('currentRatio', np.nan)
        price_book = info.get('priceToBook', np.nan)
        roe = info.get('returnOnEquity', np.nan) * 100  # Convert to percentage
        roa = info.get('returnOnAssets', np.nan) * 100    # Convert to percentage
        interest_coverage = get_interest_coverage(ticker)
        dividend_growth = info.get('dividendRate', np.nan)  # Simplified
        eps_growth = info.get('earningsQuarterlyGrowth', np.nan)

        # Book value and EPS growth could be more complex to calculate
        # Here we'll use simplified placeholders
        book_value_growth = info.get('bookValue', np.nan)  # Placeholder
        eps = info.get('earningsPerShare', np.nan)         # Latest EPS

        # Economic moat is not directly available; using 'moat' information if available
        moat = get_moat_stocks(ticker)

        return {
            'Ticker': ticker,
            'Debt/Equity': debt_equity,
            'Current Ratio': current_ratio,
            'Price/Book': price_book,
            'ROE (%)': roe,
            'ROA (%)': roa,
            'Interest Coverage': interest_coverage,
            'Moat': moat,
            'EPS Growth': eps_growth,
            # Additional fields can be added as needed
        }
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return None

In [60]:
# Scoring functions
def score_metric(value, thresholds, scores):
    if pd.isna(value):
        return 0
    for i in range(len(thresholds)):
        if value >= thresholds[i]:
            return scores[i]
    return 0

In [64]:
financial_data = []
for ticker in sp500_tickers:
    data = get_financials(ticker)
    if data:
        financial_data.append(data)
df = pd.DataFrame(financial_data)


filtered_df = df[
    (df['Debt/Equity'] < 0.5) &
    (df['Current Ratio'] > 1.5) & (df['Current Ratio'] < 2.5) &
    (df['Price/Book'] < 1.5) &
    (df['ROE (%)'] > 8) &
    (df['ROA (%)'] > 6) &
    (df['Interest Coverage'] > 5) &
    (df['Moat'] > 0) &
    (df['EPS Growth'] > 0.08)
]



Necessary financial data not available for AFL.
Necessary financial data not available for ALGN.
Necessary financial data not available for ALL.
Necessary financial data not available for AMTM.
Necessary financial data not available for AXP.
Necessary financial data not available for AIG.
Necessary financial data not available for AMP.
Necessary financial data not available for ACGL.
Necessary financial data not available for AIZ.
Necessary financial data not available for BAC.
Necessary financial data not available for BRK.B.
Necessary financial data not available for BX.
Necessary financial data not available for BK.
Necessary financial data not available for BF.B.
Necessary financial data not available for COF.
Necessary financial data not available for SCHW.
Necessary financial data not available for CMG.
Necessary financial data not available for CB.
Necessary financial data not available for CI.
Necessary financial data not available for CINF.
Necessary financial data not availab

ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/TSCO?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=TSCO&crumb=MP%2FRhNPwyGv


Necessary financial data not available for TSCO.


ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/TSCO?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=TSCO&crumb=MP%2FRhNPwyGv
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/TT?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=TT&crumb=MP%2FRhNPwyGv


Necessary financial data not available for TT.


...

Necessary financial data not available for TRMB.


ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/TRMB?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=TRMB&crumb=MP%2FRhNPwyGv
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/TFC?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=TFC&crumb=MP%2FRhNPwyGv


Necessary financial data not available for TFC.


ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/TFC?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=TFC&crumb=MP%2FRhNPwyGv


Necessary financial data not available for USB.
Necessary financial data not available for ULTA.
Necessary financial data not available for WRB.
Necessary financial data not available for WFC.


In [65]:
df

,Ticker,Debt/Equity,Current Ratio,Price/Book,ROE (%),ROA (%),Interest Coverage,Moat,EPS Growth
0,MMM,295.441,1.431,15.540826,113.293990,9.032,-9.73,0.142857,NaN
1,AOS,7.471,1.673,5.538240,29.544002,14.337,63.12,0.142857,-0.113
2,ABT,37.589,1.597,5.045763,14.840001,6.488,10.17,0.142857,0.146
3,ABBV,1174.815,0.645,50.178726,56.407000,7.720,5.74,0.142857,-0.122
4,ACN,14.127,1.099,8.038106,26.674998,11.627,162.73,0.142857,0.227
...,...,...,...,...,...,...,...,...,...
498,XYL,19.840,1.956,2.800220,8.081000,4.429,14.86,0.142857,0.428
499,YUM,NaN,1.370,NaN,NaN,24.328,4.49,0.000000,-0.082
500,ZBRA,69.307,1.366,5.966168,11.873999,5.234,4.40,0.142857,NaN
501,ZBH,53.611,1.362,1.767021,8.713000,4.610,7.21,0.142857,0.531


In [67]:
df2 = df.dropna()
len(df2)

339

In [68]:
filtered_df

,Ticker,Debt/Equity,Current Ratio,Price/Book,ROE (%),ROA (%),Interest Coverage,Moat,EPS Growth


As of 14/11/2024 we see that no stocks pass Buffett's screening selection. Perhaps that is why he is so concentrated in cash right now.